In [9]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [10]:
#data = pd.read_csv('data.csv')
data = pd.read_csv('cian_parsing_result_1_sale_1_54_perm_23_Oct_2023_23_37_26_877192.csv', encoding='windows-1251', sep=';')
data.replace(-1, np.nan, inplace=True)
data.replace("-1", np.nan, inplace=True)

In [11]:
print(data.columns)

Index(['author', 'author_type', 'link', 'city', 'deal_type',
       'accommodation_type', 'timing', 'is_apartment', 'floor', 'floors_count',
       'rooms_count', 'total_meters', 'price_per_m2', 'price',
       'year_of_construction', 'living_meters', 'kitchen_meters', 'phone',
       'ceiling_height', 'bathroom_type', 'balcony_type', 'repair_type',
       'building_type', 'elevator_type', 'parking_type', 'window_view',
       'flat_layout', 'district', 'street', 'underground',
       'residential_complex'],
      dtype='object')


In [12]:
filtered_data = data.dropna(subset=['total_meters', 'living_meters', 'kitchen_meters'])
grouped_data = filtered_data.groupby('rooms_count').apply(lambda x: pd.Series({
    'avg_living_meters_ratio': (x['living_meters'] / x['total_meters']).mean(),
    'avg_kitchen_meters_ratio': (x['kitchen_meters'] / x['total_meters']).mean()
})).reset_index()

# Заполнение отсутствующих значений в столбце 'living_meters' на основе долей и общей площади
for index, row in grouped_data.iterrows():
    mask = (data['rooms_count'] == row['rooms_count']) & data['living_meters'].isnull()
    data.loc[mask, 'living_meters'] = data['total_meters'] * row['avg_living_meters_ratio']

# Заполнение отсутствующих значений в столбце 'kitchen_meters' на основе долей и общей площади
for index, row in grouped_data.iterrows():
    mask = (data['rooms_count'] == row['rooms_count']) & data['kitchen_meters'].isnull()
    data.loc[mask, 'kitchen_meters'] = data['total_meters'] * row['avg_kitchen_meters_ratio']

In [13]:
grouped_data

,rooms_count,avg_living_meters_ratio,avg_kitchen_meters_ratio
0,1,0.502995,0.215272


In [14]:
median_year = data['year_of_construction'].dropna().median()
data.loc[data['year_of_construction'].isnull(), 'year_of_construction'] = median_year

In [15]:
data = data.dropna(subset=['rooms_count'])

In [16]:
data['timing'] = data['timing'].apply(lambda x: 365 + x if x < 0 else x)

In [17]:
data['author_type'] = data['author_type'].map({
    'real_estate_agent': 'realtor',
    'realtor': 'realtor',
    'homeowner': 'homeowner',
    'unknown': 'unknown',
    'official_representative': 'developer',
    'developer': 'developer',
    'representative_developer': 'developer'
}).fillna('unknown')

# 2. Применение one-hot encoding
data_encoded = pd.get_dummies(data['author_type'], prefix='author')
data = pd.concat([data, data_encoded], axis=1)
data.drop(['author_type'], axis=1, inplace=True)


In [18]:
data['ceiling_height'] = data['ceiling_height'].str.replace(',', '.').str.extract('(\d+\.\d+)').astype(float)
median_ceiling_height = data['ceiling_height'].dropna().median()
data['ceiling_height'].fillna(median_ceiling_height, inplace=True)

In [19]:
data['elevator_type_missing'] = data['elevator_type'].isnull().astype(int)
data['elevator_type'].fillna('0', inplace=True)

data['passenger_elevators'] = data['elevator_type'].apply(lambda x: sum(int(num) for num in x.split() if num.isdigit() and ('пассажир' in x)))
data['cargo_elevators'] = data['elevator_type'].apply(lambda x: sum(int(num) for num in x.split() if num.isdigit() and ('груз' in x)))

# Удалите исходный столбец 'elevator_type'
data.drop(columns=['elevator_type'], inplace=True)
data = data[(data['passenger_elevators'] <= 20) & (data['cargo_elevators'] <= 20)]

In [20]:
data['bathroom_type_missing'] = data['bathroom_type'].isnull().astype(int)
data['bathroom_type'].fillna('0', inplace=True)

data['combined_bathrooms'] = data['bathroom_type'].apply(lambda x: sum(int(num) for num in x.split() if num.isdigit() and ('совмещен' in x)))
data['separate_bathrooms'] = data['bathroom_type'].apply(lambda x: sum(int(num) for num in x.split() if num.isdigit() and ('раздельн' in x)))

# Удалите исходный столбец 'bathroom_type'
data.drop(columns=['bathroom_type'], inplace=True)

In [21]:
data['balcony_type_missing'] = data['balcony_type'].isnull().astype(int)
data['balcony_type'].fillna('0', inplace=True)

# Извлеките количество балконов и лоджий из 'balcony_type'
data['balconies'] = data['balcony_type'].apply(lambda x: int(x.split()[0]) if "балкон" in str(x) else 0)
data['loggias'] = data['balcony_type'].apply(lambda x: int(x.split()[0]) if "лоджия" in str(x) else 0)

# Удалите исходный столбец 'balcony_type'
data.drop(columns=['balcony_type'], inplace=True)

In [22]:
repair_type_dummies = pd.get_dummies(data["repair_type"], prefix="repair_type")
data = pd.concat([data, repair_type_dummies], axis=1)
data.drop("repair_type", axis=1, inplace=True)

In [23]:
building_type_dummies = pd.get_dummies(data["building_type"], prefix="building_type")
data = pd.concat([data, building_type_dummies], axis=1)
data.drop("building_type", axis=1, inplace=True)

In [24]:
parking_type_dummies = pd.get_dummies(data["parking_type"], prefix="parking_type")
data = pd.concat([data, parking_type_dummies], axis=1)
data.drop("parking_type", axis=1, inplace=True)

In [25]:
window_type_dummies = pd.get_dummies(data["window_view"], prefix="window_view")
data = pd.concat([data, window_type_dummies], axis=1)
data.drop("window_view", axis=1, inplace=True)

In [26]:
districts = ['Дзержинский', 'Индустриальный', 'Кировский', 'Ленинский', 'Мотовилихинский', 'Орджоникидзевский', 'Свердловский', np.nan]
#districts = ['Дзержинский', 'Индустриальный', 'Кировский', 'Ленинский', 'Мотовилихинский', 'Орджоникидзевский', 'Свердловский']
data = data.loc[data['district'].isin(districts)]

,author,link,city,deal_type,accommodation_type,timing,is_apartment,floor,floors_count,rooms_count,total_meters,price_per_m2,price,year_of_construction,living_meters,kitchen_meters,phone,ceiling_height,flat_layout,district,street,underground,residential_complex,author_developer,author_homeowner,author_realtor,author_unknown,elevator_type_missing,passenger_elevators,cargo_elevators,bathroom_type_missing,combined_bathrooms,separate_bathrooms,balcony_type_missing,balconies,loggias,repair_type_Без ремонта,repair_type_Евроремонт,repair_type_Косметический,building_type_Блочный,building_type_Кирпичный,building_type_Монолитный,building_type_Панельный,parking_type_Наземная,window_view_Во двор,window_view_На улицу,window_view_На улицу и двор
0,Сбербанк,https://perm.cian.ru/sale/flat/291485382/,Пермь,sale,flat,0,False,4,5,1,26.0,53238,1384200,1994,18.000000,5.597070,79120211654,2.70,NaN,NaN,NaN,NaN,NaN,False,False,True,False,1,0,0,0,1,0,1,0,0,False,False,True,False,True,False,False,False,False,False,False
1,Самолет Плюс,https://perm.cian.ru/sale/flat/292418262/,Пермь,sale,flat,0,False,2,27,1,27.0,92592,2500000,2016,13.000000,5.000000,79824807712,2.70,NaN,Орджоникидзевский,Вильямса,NaN,Журавли,False,False,True,False,0,2,2,0,1,0,0,0,1,False,False,True,False,False,False,False,False,True,False,False
2,Своё Жильё,https://perm.cian.ru/sale/flat/287711385/,Пермь,sale,flat,0,False,3,9,1,25.7,141634,3640000,2023,12.000000,5.000000,79194809954,2.70,NaN,Мотовилихинский,Сапфирная,NaN,Погода,False,False,True,False,0,1,0,0,1,0,0,0,1,False,False,True,False,False,False,False,False,False,True,False
3,RedHome,https://perm.cian.ru/sale/flat/289457307/,Пермь,sale,flat,0,False,8,9,1,28.0,75714,2120000,1991,13.000000,7.000000,79194403802,2.70,NaN,Орджоникидзевский,Памирская,NaN,NaN,False,False,True,False,0,1,0,0,1,0,0,1,0,False,False,True,False,False,False,False,False,False,True,False
4,Своё Жильё,https://perm.cian.ru/sale/flat/290315038/,Пермь,sale,flat,0,False,4,9,1,25.7,138910,3570000,2023,12.000000,5.000000,79194809954,2.70,NaN,Мотовилихинский,Сапфирная,NaN,Погода,False,False,True,False,0,1,0,0,1,0,0,0,1,False,False,True,False,False,False,False,False,False,True,False
5,ID 102727526,https://perm.cian.ru/sale/flat/292321258/,Пермь,sale,flat,45,False,1,5,1,38.5,75868,2920923,2020,14.000000,9.000000,79194803811,2.75,NaN,NaN,NaN,NaN,NaN,False,False,True,False,1,0,0,0,0,1,0,0,1,False,False,True,False,False,False,False,True,False,True,False
6,ID 105393952,https://perm.cian.ru/sale/flat/293592137/,Пермь,sale,flat,0,False,4,6,1,34.8,96264,3350000,2019,16.000000,10.000000,79194803466,2.70,NaN,Мотовилихинский,Лядовская,NaN,Вишневый,False,True,False,False,0,1,0,0,2,2,0,1,0,False,False,True,False,True,False,False,False,False,True,False
7,ID 73920272,https://perm.cian.ru/sale/flat/293415438/,Пермь,sale,flat,0,False,2,10,1,41.0,121951,5000000,2007,23.000000,9.000000,79194405510,2.65,NaN,Мотовилихинский,Хрустальная,NaN,NaN,False,True,False,False,0,1,0,0,1,0,1,0,0,False,False,True,False,False,False,False,True,True,False,False
8,ID 102727526,https://perm.cian.ru/sale/flat/287232140/,Пермь,sale,flat,0,False,3,5,1,37.7,82193,3098679,2022,17.000000,9.000000,79194803811,2.75,NaN,NaN,NaN,NaN,NaN,False,False,True,False,1,0,0,0,0,1,0,0,1,False,False,True,True,False,False,False,True,False,True,False
9,ID 102727526,https://perm.cian.ru/sale/flat/292383140/,Пермь,sale,flat,0,False,5,5,1,79.0,69391,5481901,2021,43.000000,12.000000,79194803811,2.75,NaN,NaN,NaN,NaN,NaN,False,False,True,False,1,0,0,0,0,1,0,0,1,False,False,True,False,False,False,False,True,True,False,False


In [27]:
price_aggregation = data.groupby('is_apartment')['price'].agg(['mean', 'median', 'min', 'max', 'std', 'count'])
price_aggregation

,mean,median,min,max,std,count
is_apartment,,,,,,
False,3.090249e+06,2990000.0,720000,6850000,1.281558e+06,39


In [28]:
district_dummies = pd.get_dummies(data["district"], prefix="district")
data = pd.concat([data, district_dummies], axis=1)
data.drop("district", axis=1, inplace=True)

In [29]:
data['residential_complex'] = data['residential_complex'].notna()

In [30]:
columns_to_drop = ['author', 'deal_type', 'accommodation_type', 'phone', 'flat_layout', 'street', 'underground', 'city']

In [31]:
data.drop(columns=columns_to_drop, inplace=True)

In [32]:
data.columns
# Осталось district

Index(['link', 'timing', 'is_apartment', 'floor', 'floors_count',
       'rooms_count', 'total_meters', 'price_per_m2', 'price',
       'year_of_construction', 'living_meters', 'kitchen_meters',
       'ceiling_height', 'residential_complex', 'author_developer',
       'author_homeowner', 'author_realtor', 'author_unknown',
       'elevator_type_missing', 'passenger_elevators', 'cargo_elevators',
       'bathroom_type_missing', 'combined_bathrooms', 'separate_bathrooms',
       'balcony_type_missing', 'balconies', 'loggias',
       'repair_type_Без ремонта', 'repair_type_Евроремонт',
       'repair_type_Косметический', 'building_type_Блочный',
       'building_type_Кирпичный', 'building_type_Монолитный',
       'building_type_Панельный', 'parking_type_Наземная',
       'window_view_Во двор', 'window_view_На улицу',
       'window_view_На улицу и двор', 'district_Дзержинский',
       'district_Индустриальный', 'district_Кировский',
       'district_Мотовилихинский', 'district_Орджоникид

In [33]:
data

,link,timing,is_apartment,floor,floors_count,rooms_count,total_meters,price_per_m2,price,year_of_construction,living_meters,kitchen_meters,ceiling_height,residential_complex,author_developer,author_homeowner,author_realtor,author_unknown,elevator_type_missing,passenger_elevators,cargo_elevators,bathroom_type_missing,combined_bathrooms,separate_bathrooms,balcony_type_missing,balconies,loggias,repair_type_Без ремонта,repair_type_Евроремонт,repair_type_Косметический,building_type_Блочный,building_type_Кирпичный,building_type_Монолитный,building_type_Панельный,parking_type_Наземная,window_view_Во двор,window_view_На улицу,window_view_На улицу и двор,district_Дзержинский,district_Индустриальный,district_Кировский,district_Мотовилихинский,district_Орджоникидзевский,district_Свердловский
0,https://perm.cian.ru/sale/flat/291485382/,0,False,4,5,1,26.0,53238,1384200,1994,18.000000,5.597070,2.70,False,False,False,True,False,1,0,0,0,1,0,1,0,0,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False
1,https://perm.cian.ru/sale/flat/292418262/,0,False,2,27,1,27.0,92592,2500000,2016,13.000000,5.000000,2.70,True,False,False,True,False,0,2,2,0,1,0,0,0,1,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False
2,https://perm.cian.ru/sale/flat/287711385/,0,False,3,9,1,25.7,141634,3640000,2023,12.000000,5.000000,2.70,True,False,False,True,False,0,1,0,0,1,0,0,0,1,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False
3,https://perm.cian.ru/sale/flat/289457307/,0,False,8,9,1,28.0,75714,2120000,1991,13.000000,7.000000,2.70,False,False,False,True,False,0,1,0,0,1,0,0,1,0,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False
4,https://perm.cian.ru/sale/flat/290315038/,0,False,4,9,1,25.7,138910,3570000,2023,12.000000,5.000000,2.70,True,False,False,True,False,0,1,0,0,1,0,0,0,1,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False
5,https://perm.cian.ru/sale/flat/292321258/,45,False,1,5,1,38.5,75868,2920923,2020,14.000000,9.000000,2.75,False,False,False,True,False,1,0,0,0,0,1,0,0,1,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,False,False
6,https://perm.cian.ru/sale/flat/293592137/,0,False,4,6,1,34.8,96264,3350000,2019,16.000000,10.000000,2.70,True,False,True,False,False,0,1,0,0,2,2,0,1,0,False,False,True,False,True,False,False,False,False,True,False,False,False,False,True,False,False
7,https://perm.cian.ru/sale/flat/293415438/,0,False,2,10,1,41.0,121951,5000000,2007,23.000000,9.000000,2.65,False,False,True,False,False,0,1,0,0,1,0,1,0,0,False,False,True,False,False,False,False,True,True,False,False,False,False,False,True,False,False
8,https://perm.cian.ru/sale/flat/287232140/,0,False,3,5,1,37.7,82193,3098679,2022,17.000000,9.000000,2.75,False,False,False,True,False,1,0,0,0,0,1,0,0,1,False,False,True,True,False,False,False,True,False,True,False,False,False,False,False,False,False
9,https://perm.cian.ru/sale/flat/292383140/,0,False,5,5,1,79.0,69391,5481901,2021,43.000000,12.000000,2.75,False,False,False,True,False,1,0,0,0,0,1,0,0,1,False,False,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False


In [34]:
data.to_csv('data_for_model.csv', index=False)